In this notebook we explore different ways of adding noise to the circuits.

In [1]:
import numpy as np
import sympy
import cirq
import tensorflow_quantum as tfq
from tqdm import tqdm
import tensorflow as tf

from utilities.variational import VQE
from utilities.circuit_basics import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer import UnitaryMurder

n_qubits = 4
qlr = 0.01
qepochs = 10**3
verbose=0
g=1
J=0
noise=0.0
problem="TFIM"
vqe_handler = VQE(n_qubits=n_qubits, lr=qlr, epochs=qepochs, patience=100,
                  random_perturbations=True, verbose=verbose, g=g, J = J, noise=noise, problem=problem)

iid = IdInserter(n_qubits=n_qubits)
Simp = Simplifier(n_qubits=n_qubits)
killer = UnitaryMurder(vqe_handler)
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]

In [13]:
circuit, symbols, idx_symbols = vqe_handler.give_circuit(indexed_circuit) 
circuit

(0, 0): ───Rx(th_0)───

(0, 1): ───Rx(th_1)───

(0, 2): ───Rx(th_2)───

(0, 3): ───Rx(th_3)───

In [16]:
circuit.with_noise(cirq.depolarize(0.01))

(0, 0): ───Rx(th_0)───D(0.01)[<virtual>]───

(0, 1): ───Rx(th_1)───D(0.01)[<virtual>]───

(0, 2): ───Rx(th_2)───D(0.01)[<virtual>]───

(0, 3): ───Rx(th_3)───D(0.01)[<virtual>]───

In [14]:
circuit_res= cirq.resolve_parameters(circuit, {s:0 for s,k in zip(symbols, range(len(symbols)))})
tfq_circuit_noise = tfq.convert_to_tensor([circuit_res])
noisy_expectation = tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.01)))
tfq_layer_noisy = noisy_expectation(tfq_circuit_noise, operators=tfq.convert_to_tensor([vqe_handler.observable]))
noisy_energy = np.squeeze(tf.math.reduce_sum(tfq_layer_noisy, axis=-1))
noisy_energy

array(-3.9466667, dtype=float32)

In [17]:
pauls = [cirq.X, cirq.Y, cirq.Z]
circuit_res = cirq.resolve_parameters(circuit, {s:0 for s,k in zip(symbols, range(len(symbols)))})
p=0.01
cbatch = []
totals=10**3
for k in range(totals):
    circ = circuit_res.copy()
    
    if k<int((1-p)*totals):
        cbatch.append(circ)
    elif int((1-p)*totals)<=k:
        which = np.random.choice(range(3), 1)[0]
        gate = pauls[which]
        for q in vqe_handler.qubits:
            circ.append(gate.on(q))
        cbatch.append(circ)

In [19]:
tfq_circuit = tfq.convert_to_tensor(cbatch)
expectation = tfq.layers.Expectation()
tfq_layer = expectation(tfq_circuit, operators=tfq.convert_to_tensor([vqe_handler.observable]*1000))
averaged_unitaries = tf.math.reduce_mean(tfq_layer, axis=0)
energy = np.squeeze(tf.math.reduce_sum(averaged_unitaries, axis=-1))
energy

array(-3.944, dtype=float32)